In [2]:
pip install pygame

Note: you may need to restart the kernel to use updated packages.


In [32]:
import pygame
import random

# Initialize Pygame
pygame.init()

# Screen dimensions
SCREEN_WIDTH = 400
SCREEN_HEIGHT = 600
SCREEN = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Car Jump Over Obstacles")

# Colors (used for text or other elements, not for drawing the player or obstacles)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)

# Game parameters
PLAYER_WIDTH = 50
PLAYER_HEIGHT = 80
OBSTACLE_WIDTH = 50
OBSTACLE_HEIGHT = 50
LANE_WIDTH = SCREEN_WIDTH // 3

# Load images (make sure these PNG files exist in your project directory)
PLAYER_IMAGE = pygame.image.load("C:/Users/adity/Downloads/pngtree-car-top-view-image-png-image_8931232.png")  # Add the path to your player image
OBSTACLE_IMAGE = pygame.image.load("C:/Users/adity/Downloads/165649502402200-transport-car-police.png")  # Add the path to your obstacle image
BACKGROUND_IMAGE = pygame.image.load("C:/Users/adity/Downloads/road-illustration-white-background-lane-131018520.webp")  # Add the path to your background image

# Resize images if necessary
PLAYER_IMAGE = pygame.transform.scale(PLAYER_IMAGE, (PLAYER_WIDTH, PLAYER_HEIGHT))
OBSTACLE_IMAGE = pygame.transform.scale(OBSTACLE_IMAGE, (OBSTACLE_WIDTH, OBSTACLE_HEIGHT))

class Player(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = PLAYER_IMAGE
        self.rect = self.image.get_rect()
        self.rect.centerx = SCREEN_WIDTH // 2
        self.rect.bottom = SCREEN_HEIGHT - 50
        self.lane = 1  # Start in middle lane (0: left, 1: middle, 2: right)
        self.jumping = False
        self.jump_count = 10
        self.jump_velocity = 0  # For smooth jumping behavior

    def update(self):
        if self.jumping:
            self.jump_velocity += 1  # Gravity
            self.rect.y += self.jump_velocity
            if self.rect.bottom >= SCREEN_HEIGHT - 50:  # If it hits the ground
                self.rect.bottom = SCREEN_HEIGHT - 50
                self.jumping = False
                self.jump_velocity = 0

    def jump(self):
        if not self.jumping:
            self.jumping = True
            self.jump_velocity = -15  # Set initial jump speed


class Obstacle(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = OBSTACLE_IMAGE
        self.rect = self.image.get_rect()
        self.lane = random.randint(0, 2)
        self.rect.x = LANE_WIDTH * self.lane + (LANE_WIDTH - OBSTACLE_WIDTH) // 2
        self.rect.y = -OBSTACLE_HEIGHT

    def update(self):
        self.rect.y += 5  # Move obstacle down
        if self.rect.top > SCREEN_HEIGHT:  # Remove obstacles that go off screen
            self.kill()


class CarGame:
    def __init__(self):
        self.screen = SCREEN
        self.clock = pygame.time.Clock()
        self.font = pygame.font.Font(None, 36)
        self.reset_game()

        # Background scrolling
        self.bg_y1 = 0
        self.bg_y2 = -SCREEN_HEIGHT

        # Timer for time-based score increase
        self.time_score_timer = 0

    def reset_game(self):
        # Sprite groups
        self.all_sprites = pygame.sprite.Group()
        self.obstacles = pygame.sprite.Group()

        # Create player
        self.player = Player()
        self.all_sprites.add(self.player)

        # Game state
        self.score = 0
        self.game_over = False
        self.speed = 5  # Initial speed
        self.spawn_timer = 0

    def spawn_obstacle(self):
        obstacle = Obstacle()
        self.all_sprites.add(obstacle)
        self.obstacles.add(obstacle)

    def handle_events(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return False

            if event.type == pygame.KEYDOWN and not self.game_over:
                # Lane switching
                if event.key == pygame.K_LEFT and self.player.lane > 0:
                    self.player.lane -= 1
                    self.player.rect.x = LANE_WIDTH * self.player.lane + (LANE_WIDTH - PLAYER_WIDTH) // 2

                if event.key == pygame.K_RIGHT and self.player.lane < 2:
                    self.player.lane += 1
                    self.player.rect.x = LANE_WIDTH * self.player.lane + (LANE_WIDTH - PLAYER_WIDTH) // 2

                # Jumping
                if event.key == pygame.K_SPACE:
                    self.player.jump()

        # Restart game
        keys = pygame.key.get_pressed()
        if self.game_over and keys[pygame.K_r]:
            self.reset_game()

        return True

    def update(self):
        if self.game_over:
            return

        # Spawn obstacles
        self.spawn_timer += 1
        if self.spawn_timer > 60:  # Adjust for obstacle frequency
            self.spawn_obstacle()
            self.spawn_timer = 0

        # Update sprites
        self.all_sprites.update()

        # Collision detection (only if not jumping)
        if not self.player.jumping:
            hits = pygame.sprite.spritecollide(self.player, self.obstacles, False)
            if hits:
                self.game_over = True

        # Increase score by clearing obstacles
        for obstacle in self.obstacles:
            if obstacle.rect.top > SCREEN_HEIGHT:
                obstacle.kill()
                self.score += 1

        # Increase speed every 5 points
        if self.score % 5 == 0 and self.speed < 10:  # You can change the speed increase condition
            self.speed += 1

        # Increase score by time
        self.time_score_timer += 1
        if self.time_score_timer >= 60:  # Increase score every 1 second
            self.score += 1
            self.time_score_timer = 0  # Reset timer

    def draw_background(self):
        # Scrolling background
        self.screen.blit(BACKGROUND_IMAGE, (0, self.bg_y1))
        self.screen.blit(BACKGROUND_IMAGE, (0, self.bg_y2))

        self.bg_y1 += self.speed
        self.bg_y2 += self.speed

        # Reset background positions for scrolling effect
        if self.bg_y1 >= SCREEN_HEIGHT:
            self.bg_y1 = -SCREEN_HEIGHT
        if self.bg_y2 >= SCREEN_HEIGHT:
            self.bg_y2 = -SCREEN_HEIGHT

    def draw(self):
        # Draw background
        self.draw_background()

        # Draw lane markers
        for i in range(1, 3):
            pygame.draw.line(self.screen, WHITE, (i * LANE_WIDTH, 0), (i * LANE_WIDTH, SCREEN_HEIGHT), 3)

        # Draw sprites
        self.all_sprites.draw(self.screen)

        # Draw score
        score_text = self.font.render(f"Score: {self.score}", True, BLACK)
        self.screen.blit(score_text, (10, 10))

        # Game over screen
        if self.game_over:
            game_over_text = self.font.render("Game Over! Press R to Restart", True, RED)
            text_rect = game_over_text.get_rect(center=(SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2))
            self.screen.blit(game_over_text, text_rect)

        pygame.display.flip()

    def run(self):
        running = True
        while running:
            # Handle events
            running = self.handle_events()

            # Update game state
            self.update()

            # Draw everything
            self.draw()

            # Control game speed
            self.clock.tick(60)

        pygame.quit()


# Run the game
if __name__ == "__main__":
    game = CarGame()
    game.run()
